# Imports

In [1]:
import os
import sys

In [2]:
current_cwd = os.getcwd()
src_path = '/'.join(current_cwd.split('/')[:-1])
sys.path.append(src_path)

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy import linalg
from scipy.linalg import sqrtm
from scipy.stats import entropy
from torch.nn.functional import adaptive_avg_pool2d
from tqdm.auto import tqdm

from src.generator.model import Generator
from src.encoders.text_encoder import RNNEncoder

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Inception model

In [5]:
class InceptionV3(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = f'cuda:{0}' if torch.cuda.is_available() else 'cpu'
        self.model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True).to(self.device)
        print(self.model.fc)
        self.linear = self.model.fc
        self.model.fc, self.model.dropout = [nn.Sequential()] * 2
      
    @torch.no_grad()
    def get_last_layer(self, x):
        x = F.interpolate(x, size=300, mode='bilinear', align_corners=False, recompute_scale_factor=False)
        return self.model(x)

In [6]:
classifier = InceptionV3().to(device)
classifier = classifier.eval()

# Dataset + DataLoader

In [7]:
from torch.utils.data import DataLoader
from torchvision import transforms

from src.objects.dataset import AttnGANDataset

In [8]:
batch_size = 8
split_dir, bshuffle = 'test', True
image_size = 64 * (2 ** (3 - 1))

image_transform = transforms.Compose([
    transforms.Scale(int(image_size * 76 / 64)),
    transforms.RandomCrop(image_size),
    transforms.RandomHorizontalFlip()
])

data_dir = "../data"
dataset = AttnGANDataset(data_dir, split_dir, image_transform)

test_loader = DataLoader(dataset, batch_size=batch_size, drop_last=True, shuffle=bshuffle)
n_words = test_loader.dataset.n_words

# Generator + Text Encoder

In [9]:
generator = Generator().to(device)
generator.load_state_dict(torch.load("../gen_weights_70/gen_weights_epoch_9.pth", map_location=device))
generator = generator.eval()

In [10]:
text_encoder = RNNEncoder.load("../encoder_weights/text_encoder200.pth", n_words)
text_encoder.to(device)

for p in text_encoder.parameters():
    p.requires_grad = False
text_encoder = text_encoder.eval()

# FID calculation

In [11]:
def calculate_fid(repr1, repr2):
    # shape of reprs: (-1, embed_dim)
    
    # shape of mus: (embed_dim, )
    mu_r, mu_g = np.mean(repr1, axis=0), np.mean(repr2, axis=0)
    # rowvar=False:
    #     each column represents a variable, while the rows contain observations
    # shape of sigmas: (embed_dim, embed_dim)
    sigma_r, sigma_g = np.cov(repr1, rowvar=False), np.cov(repr2, rowvar=False)
    
    diff = mu_r - mu_g
    diff_square_norm = diff.dot(diff)
    
    product = sigma_r.dot(sigma_g)
    sqrt_product, _ = sqrtm(product, disp=False)
    
    # np.isfinite:
    #     Test element-wise for finiteness 
    #     (not infinity and not Not a Number)
    if not np.isfinite(sqrt_product).all():
        eye_matrix = np.eye(sigma_r.shape[0]) * 1e-8
        sqrt_product = linalg.sqrtm((sigma_r + eye_matrix).dot(sigma_g + eye_matrix))
    
    # np.iscomplexobj:
    #     Check for a complex type or an array of complex numbers.
    #     The return value, True if x is of a complex type
    #     or has at least one complex element.
    if np.iscomplexobj(sqrt_product):
        sqrt_product = sqrt_product.real

    fid = diff_square_norm + np.trace(sigma_r + sigma_g - 2 * sqrt_product)
    
    return fid

In [12]:
from src.objects.utils import prepare_data

In [13]:
def build_representations():
    real_reprs = np.zeros((len(test_loader) * batch_size, 2048))
    fake_reprs = np.zeros((len(test_loader) * batch_size, 2048))
    
    for i, batch in enumerate(tqdm(test_loader, desc="Build representations")):
        images, captions, captions_len, cls_ids, file_names = prepare_data(batch, device)
        
        word_embeds, sentence_embeds = text_encoder(captions, captions_len)
    
        mask = (captions == 0)
        num_words = word_embeds.size(2)

        if mask.size(1) > num_words:
            mask = mask[:, :num_words]

        noise = torch.randn(sentence_embeds.shape[0], 100, device=device)
        fake_images, _, mu, log_var = generator(noise, sentence_embeds, word_embeds, mask)
        
        fake_images = fake_images[2]

        clf_out_real = classifier.get_last_layer(images[0])
        clf_out_fake = classifier.get_last_layer(fake_images)


        real_reprs[i * batch_size: (i + 1) * batch_size] = clf_out_real.cpu()
        fake_reprs[i * batch_size: (i + 1) * batch_size] = clf_out_fake.cpu()
            
    return real_reprs, fake_reprs

## Build representations

In [14]:
import warnings

warnings.filterwarnings("ignore")

In [16]:
real_values, fake_values = build_representations()

## FID value

In [17]:
fid_value = calculate_fid(real_values, fake_values)
print(f"FID value = {fid_value}")

# Inception score

In [18]:
def inception_score(reprs, batch_size):
    def get_pred(x):
        x = classifier.linear(torch.tensor(x, dtype=torch.float, device=device))
        return F.softmax(x).data.cpu().numpy()


    preds = np.zeros((reprs.shape[0], 1000))
    
    splits = 0
    for i in range(0, len(preds), batch_size):
        z = get_pred(reprs[i:i + batch_size])
        preds[i:i + batch_size] = z
        splits += 1
    
    split_scores = []

    for k in range(splits):
        part = preds[k * batch_size: (k+1) * batch_size, :]
        py = np.mean(part, axis=0)
        
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
            
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

In [19]:
inception_score(fake_values, batch_size)